In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 
from collections import OrderedDict

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

### 0과 1이 아닌 확률로 예측

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs 
            preds = preds.astype(float)
            predictions += preds.tolist()
    return predictions

# model1 

### model 학습에 사용한 정보 load

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':3e-3,
    'BATCH_SIZE': 128,
    'SEED':2023
}

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(CFG['SEED'])

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)
    
fc = nn.Sequential(OrderedDict([
('bn1', nn.BatchNorm1d(1000)),
('drop1', nn.Dropout(p=0.5)),
('linear1', nn.Linear(1000, 256)),
('drop2', nn.Dropout(p=0.5)),
('linear2', nn.Linear(256, 64)),
('drop3', nn.Dropout(p=0.5)),
('linear3', nn.Linear(64, 10))
]))

class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.backbone = models.resnext50_32x4d(pretrained=True)
        for param in self.backbone.parameters():
            param.requiresGrad = False
        self.fc = fc
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.fc(x))
        return x

In [9]:
model1 = BaseModel().to(device)
model1.load_state_dict(torch.load("/model1.pth"))

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

<All keys matched successfully>

In [ ]:
test = pd.read_csv('/dacon-blocks/test.csv')
test['img_path'] = test['img_path'].apply(lambda x: '/dacon-blocks/test_whitegrid' + x[6:])

test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=7)

first_preds = inference(model1, test_loader, device)

In [16]:
first_preds[2]

[0.162592813372612,
 1.0,
 8.007432916201651e-05,
 0.010709221474826336,
 1.0,
 6.096554017176459e-08,
 0.007736776024103165,
 1.189292743219994e-05,
 0.9999997615814209,
 0.9999940395355225]

# model2

### model 학습에 사용한 정보 load

In [17]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':3e-3,
    'BATCH_SIZE': 128,
    'SEED':2023
}

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(CFG['SEED'])

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

fc = nn.Sequential(OrderedDict([
('bn1', nn.BatchNorm1d(1000)),
('drop1', nn.Dropout(p=0.5)),
('linear1', nn.Linear(1000, 256)),
('drop2', nn.Dropout(p=0.5)),
('linear2', nn.Linear(256, 64)),
('drop3', nn.Dropout(p=0.5)),
('linear3', nn.Linear(64, 10))
]))

class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.backbone = models.resnext50_32x4d(pretrained=True)
        for param in self.backbone.parameters():
            param.requiresGrad = False
        
        self.fc = fc
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.fc(x))
        return x

In [18]:
model2 = BaseModel().to(device)
model2.load_state_dict(torch.load("model2.pth"))

<All keys matched successfully>

In [19]:
test = pd.read_csv('/dacon-blocks/test.csv')
test['img_path'] = test['img_path'].apply(lambda x: '/dacon-blocks/test_whitegrid' + x[6:])

test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=7)

second_preds = inference(model2, test_loader, device)

  0%|          | 0/12 [00:00<?, ?it/s]

In [23]:
second_preds[2]

[0.09511613845825195,
 0.9953898191452026,
 1.3383831173996441e-05,
 1.7493347286290373e-06,
 1.0,
 0.017939502373337746,
 0.000358765508281067,
 2.7170221983396914e-07,
 0.9996417760848999,
 0.999885082244873]

# model3

### model 학습에 사용한 정보 load

In [24]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'PATIENCE': 10,
    'LEARNING_RATE':4e-4,
    'BATCH_SIZE': 128,
    'SEED':2023
}

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0, 0, 0), std=(1,1,1), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
model3 = torch.load("model3.pth")

In [ ]:
test = pd.read_csv('/dacon-blocks/test.csv')
test['img_path'] = test['img_path'].apply(lambda x: '/dacon-blocks/test_whitegrid' + x[6:])

test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=7)

third_preds = inference(model3, test_loader, device)

In [29]:
third_preds[2]

[0.9999905824661255,
 1.0,
 2.0618767848645803e-07,
 2.2143453737299978e-10,
 0.9800875782966614,
 8.750817670488686e-08,
 0.9690304398536682,
 2.0526066180082125e-07,
 0.0002022561093326658,
 1.0]

# model4

### model 학습에 사용한 정보 load

In [ ]:
test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0, 0, 0), std=(1,1,1), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

from collections import OrderedDict
fc = nn.Sequential(OrderedDict([
('bn1', nn.BatchNorm1d(1000)),
('drop1', nn.Dropout(p=0.5)),
('linear1', nn.Linear(1000, 256)),
('drop2', nn.Dropout(p=0.5)),
('linear2', nn.Linear(256, 64)),
('drop3', nn.Dropout(p=0.5)),
('linear3', nn.Linear(64, 10))
]))

class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.backbone = models.resnext50_32x4d(pretrained=True)
        for param in self.backbone.parameters():
            param.requiresGrad = False
        
        self.fc = fc
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.fc(x))
        return x

In [ ]:
model4 = torch.load("model4.pth")

In [ ]:
test = pd.read_csv('/dacon-blocks/test.csv')
test['img_path'] = test['img_path'].apply(lambda x: '/dacon-blocks/test_whitegrid' + x[6:])

test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=7)

fourth_preds = inference(model4, test_loader, device)

In [ ]:
fourth_preds[2]

# ensemble

In [30]:
#값을 계산하기 위해 np.array 실행
a = np.array(first_preds)
b = np.array(second_preds)
c = np.array(third_preds)
d = np.array(fourth_preds)

In [31]:
a[0]

array([1.93780512e-01, 9.97888029e-01, 2.05219499e-06, 1.02106490e-10,
       1.67343344e-11, 6.17196008e-12, 1.00000000e+00, 2.29292640e-10,
       1.00000000e+00, 1.35424864e-06])

In [32]:
#동일한 비율로 각 확률값 계산
ensem = (a*0.25 + b*0.25 +c*0.25 + d*0.25)

# 확률값이 0.5 이하면 0, 0.5 초과면 1로 설정
ensem[ensem <= 0.5] = 0
ensem[ensem > 0.5] = 1

ensem = ensem.astype(int)
ensem

array([[1.00495516e-01, 8.75524859e-01, 3.13298164e-10, ...,
        2.63947976e-14, 1.00000000e+00, 1.41544064e-10],
       [1.02709800e-01, 1.00000000e+00, 9.41041615e-10, ...,
        1.34069849e-11, 1.79601172e-02, 2.26142911e-05],
       [4.19233178e-01, 9.98463273e-01, 3.12214493e-05, ...,
        4.12329677e-06, 6.66614598e-01, 9.99959707e-01],
       ...,
       [1.21581158e-10, 4.21042800e-10, 1.00000000e+00, ...,
        1.00000000e+00, 1.11736209e-10, 1.00000000e+00],
       [2.95706623e-04, 3.35224740e-01, 3.33306392e-01, ...,
        6.13697811e-14, 1.00000000e+00, 9.87041314e-01],
       [9.99173880e-01, 1.29146659e-11, 9.81517375e-01, ...,
        5.93646744e-14, 1.08383442e-08, 1.40001147e-06]])

In [ ]:
preds = ensem.tolist()
preds[1]

In [ ]:
submit = pd.read_csv("/dacon-blocks/sample_submission.csv")
submit.iloc[:,1:] = preds
print(submit.head())
submit.to_csv('ensemble.csv', index=False)